In [1]:
import requests
from re import split
import re  
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#lists for data in dataframe(columnwise) 
a = []
b = []
c = []


df = pd.DataFrame()

for i in range(1,35):
    
    url="https://www.flipkart.com/mobiles/pr?sid=tyy%2C4io&otracker=categorytree&page="+str(i)
    
    data = requests.get(url)
    soup = BeautifulSoup(data.text,'html.parser')
    
    names = soup.find_all(class_="_4rR01T")
    price = soup.find_all(class_="_3I9_wc _27UcVY")
    details = soup.find_all(class_="_1xgFaf")
    
    
   
    for i,j,k in zip(names,price,details):
        a.append(str(i.text))
        b.append(str(j.text))
        c.append(str(k.text))
               
#data entering to dataframe    
df['Brand']=a
df['Price']=b
df['Details']=c





In [2]:
df['Price'].replace('(\W+)','',regex=True, inplace = True)#to remove rupees sign

#brand contain name of company and some details about phone(model,colour & memmory)
df[['Brand','spec']]=df.Brand.str.split(' ',1,expand=True)#when 1st ' ' traversed ,remaining data split into df['spec']
#here only brands contained in df[Brand] 

df=df.drop(['spec'],axis=1) #model,colour & memmory moved into column 'spec'....so delete that column 'spec'


#value contained before 'RAM' in df['Details'] move to df[Ram]
df[['Ram','Details']] = df.Details.str.split('RAM',1,expand=True)


#value contained before 'ROM' in df['Details'] move to df[Rom]
df[['Rom','Details']] = df.Details.str.split('ROM',1,expand=True)
df['Rom'].replace('(\W+)','',regex=True, inplace = True)



In [3]:

df['Extended_Memory']=df['Details'].str.extract(r'(\d* TB|\d* GB)') #extracting Extended_Memory from df['Details'] 



In [4]:

df['Battery_capacity']=df['Details'].str.extract(r'(\d* mAh|\d* Ah)')#extracting Battery_capacity from df['Details'] 



In [5]:
df['Details'].replace('(.* Battery)','',regex=True, inplace = True)#deleting all data upto Battery from df['Details']
#'(.* Battery)' implies : =any character upto Battery (including Battery)



In [6]:
df['Processor']=df['Details'].str.extract('(.*Processor)')


In [7]:
df

,Brand,Price,Details,Ram,Rom,Extended_Memory,Battery_capacity,Processor
0,POCO,12999,Qualcomm Snapdragon 662 ProcessorOne Year Warr...,6 GB,64GB,512 GB,6000 mAh,Qualcomm Snapdragon 662 Processor
1,REDMI,9999,MediaTek Helio G25 ProcessorBrand Warranty of ...,4 GB,64GB,512 GB,5000 mAh,MediaTek Helio G25 Processor
2,realme,10999,MediaTek Helio G85 Processor1 Year Warranty fo...,4 GB,64GB,256 GB,6000 mAh,MediaTek Helio G85 Processor
3,realme,9999,MediaTek Helio G85 Processor1 Year Warranty fo...,3 GB,32GB,256 GB,6000 mAh,MediaTek Helio G85 Processor
4,realme,10999,MediaTek Helio G85 Processor1 Year Warranty fo...,4 GB,64GB,256 GB,6000 mAh,MediaTek Helio G85 Processor
...,...,...,...,...,...,...,...,...
583,OPPO,31999,and 6 Months for Accessories,6 GB,128GB,NaN,4015 mAh,NaN
584,REDMI,34999,MediaTek Helio G80 Processor1 Year Manufacture...,4 GB,128GB,512 GB,5020 mAh,MediaTek Helio G80 Processor
585,GFive,8499,1 Year Manufacturer warranty.,32 MB,32MB,16 GB,2500 mAh,NaN
586,GFive,10349,1 Year Manufacturer warranty.,32 MB,32MB,16 GB,2500 mAh,NaN


In [15]:
df.isnull().sum()

Brand               0
Price               0
Details             0
Ram                 0
Rom                 0
Extended_Memory     0
Battery_capacity    0
Processor           0
dtype: int64

In [14]:
# drop all rows with any NaN and NaT values
df = df.dropna()
df

,Brand,Price,Details,Ram,Rom,Extended_Memory,Battery_capacity,Processor
0,POCO,12999,Qualcomm Snapdragon 662 ProcessorOne Year Warr...,6 GB,64GB,512 GB,6000 mAh,Qualcomm Snapdragon 662 Processor
1,REDMI,9999,MediaTek Helio G25 ProcessorBrand Warranty of ...,4 GB,64GB,512 GB,5000 mAh,MediaTek Helio G25 Processor
2,realme,10999,MediaTek Helio G85 Processor1 Year Warranty fo...,4 GB,64GB,256 GB,6000 mAh,MediaTek Helio G85 Processor
3,realme,9999,MediaTek Helio G85 Processor1 Year Warranty fo...,3 GB,32GB,256 GB,6000 mAh,MediaTek Helio G85 Processor
4,realme,10999,MediaTek Helio G85 Processor1 Year Warranty fo...,4 GB,64GB,256 GB,6000 mAh,MediaTek Helio G85 Processor
...,...,...,...,...,...,...,...,...
570,REDMI,679,Qualcomm Snapdragon 732G Processor1 Year Manuf...,6 GB,64GB,512 GB,5020 mAh,Qualcomm Snapdragon 732G Processor
573,realme,20990,MediaTek Helio P60 Octa Core 2.0 GHz Processor...,3 GB,32GB,256 GB,4230 mAh,MediaTek Helio P60 Octa Core 2.0 GHz Processor
580,REDMI,949,MediaTek Helio G80 Processor1 Year Manufacture...,4 GB,128GB,512 GB,5020 mAh,MediaTek Helio G80 Processor
581,Itel,20990,SC6531E Processor1 Year Manufacturer Warranty ...,4 MB,4MB,32 GB,2500 mAh,SC6531E Processor


In [16]:
df.isnull().sum()

Brand               0
Price               0
Details             0
Ram                 0
Rom                 0
Extended_Memory     0
Battery_capacity    0
Processor           0
dtype: int64

In [23]:
set(df['Brand'])

{'GIONEE',
 'Infinix',
 'Itel',
 'LAVA',
 'LG',
 'MOTOROLA',
 'Micromax',
 'Nokia',
 'POCO',
 'Panasonic',
 'REDMI',
 'Redmi',
 'SAMSUNG',
 'Salora',
 'Tecno',
 'ViVO',
 'realme'}

In [10]:
#df.to_csv('mob3.csv')

In [18]:
dd=pd.read_csv('mob1.csv')

In [19]:
dd.isnull().sum()

Unnamed: 0           0
Brand                0
Price                0
Details              0
Ram                  0
Rom                  0
Extended_Memory     13
Battery_capacity     0
Processor           18
dtype: int64

In [22]:
set(dd['Brand'])

{'BlackZone',
 'GIONEE',
 'Infinix',
 'LAVA',
 'MOTOROLA',
 'Nokia',
 'OPPO',
 'POCO',
 'REDMI',
 'SAMSUNG',
 'ViVO',
 'realme'}